In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
class NoInstance(type):
    def __call__(self, *args, **kwargs):
        raise TypeError("can't instantiate directly")


class Spam(metaclass=NoInstance):
    @staticmethod
    def grok(x):
        print(f'Spam.grok: {x}')


"""

>>> Spam.grok(42)
Spam.grok: 42
>>> s = Spam()
Traceback (most recent call last):
    ...
TypeError: can't instantiate directly
"""

doctest.testmod()

TestResults(failed=0, attempted=2)

In [4]:
class Singleton(type):
    def __init__(self, *args, **kwargs):
        self.__instance = None
        super().__init__(*args, **kwargs)

    def __call__(self, *args, **kwargs):
        if self.__instance is None:
            self.__instance = super().__call__(*args, **kwargs)

        return self.__instance


class Spam(metaclass=Singleton):
    def __init__(self):
        print('creating Spam')


"""

>>> s1 = Spam()
creating Spam
>>> s2 = Spam()
>>> s1 == s2
True
>>> s1 is s2
True
>>> s3 = Spam()
>>> s1 is s3
True
"""

doctest.testmod()

TestResults(failed=0, attempted=6)

In [5]:
import weakref


class Cached(type):
    def __init__(self, *args, **kwargs):
        self.__cache = weakref.WeakValueDictionary()
        super().__init__(*args, **kwargs)

    def __call__(self, *args):
        if args in self.__cache:
            return self.__cache[args]
        else:
            obj = super().__call__(*args)
            self.__cache[args] = obj
            return obj


class Spam(metaclass=Cached):
    def __init__(self, name):
        print('creating Spam with name: {!r}'.format(name))
        self.name = name


"""

>>> a = Spam('Guido')
creating Spam with name: 'Guido'
>>> b = Spam('Diana')
creating Spam with name: 'Diana'
>>> c = Spam('Guido')
>>> a is c
True
"""

doctest.testmod()

TestResults(failed=0, attempted=4)